<a href="https://colab.research.google.com/github/Rodrigoradzinski/Segmenta-o-de-videos/blob/main/210523TRABALHO_SEGMENTA%C3%87%C3%83O_DE_VIDEOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python

In [33]:
import numpy as np
import cv2 as cv
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import numpy as np
import cv2 as cv
import os
import tarfile                   
import tempfile                  
from six.moves import urllib     
from io import BytesIO           

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import cv2 
import IPython
from PIL import Image
from tqdm import tqdm
from tabulate import tabulate 
from sklearn.metrics import confusion_matrix

import cv2
from google.colab.patches import cv2_imshow
import cv2

import tensorflow as tf  # Adicionado
import cv2
import numpy as np
from PIL import Image
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, Image, clear_output
from io import BytesIO

from PIL import Image
import cv2
import numpy as np
from matplotlib.gridspec import GridSpec
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!wget -O 'people6.mp4' 'https://github.com/Rodrigoradzinski/Segmenta-o-de-videos/raw/main/people6.mp4'

In [34]:
def criar_colormap():
    """
    Retorna:
        Array numpy com o mapa de cores para visualizar resultados da segmentação. 
    """
    colormap = np.array([
        [128,  64, 128],
        [244,  35, 232],
        [ 70,  70,  70],
        [102, 102, 156],
        [190, 153, 153],
        [153, 153, 153],
        [250, 170,  30],
        [220, 220,   0],
        [120, 155,  42],
        [152, 251, 152],
        [ 93, 165, 227],
        [220,  20,  60],
        [255,   0,   0],
        [ 34,  34, 142],
        [  0,   0,  70],
        [  0,  60, 100],
        [  0,  80, 100],
        [  0,   0, 230],
        [119,  11,  32],
        [  0,   0,   0]], dtype=np.uint8)
    return colormap

In [35]:
LABEL_NAMES = np.asarray([
    'rua', 'faixa de pedestres', 'prédios', 'muro', 'cerca', 'poste', 'luz de trânsito',
    'sinal de trânsito', 'mato/ vegetação', 'terreno', 'céu', 'pessoa', 'motorista', 'carro', 'caminhão',
    'ônibus', 'tren', 'moto', 'bicicleta', 'vazio'])

len(LABEL_NAMES)

20

In [36]:
def executa_segmentacao(caminho):
  img_original = Image.open(caminho)
  seg_map = model.run(img_original)
  img_processada = vis_segmentacao(img_original, seg_map)
  return img_processada

In [37]:
def label_to_color_image(label):
    """
    Parâmetros:
        label: array 2D que armazena o rótulo da segmentação.

    Retorna:
        mapa de segmentação: array 2D com valores do tipo float. O elemento da array é a cor indexada pelo elemento correspondente no rótulo de input. 
        Ou seja, vai retornar uma imagem que corresponde à original, porém vai pintar os pixels com aquilo que o algoritmo previu
    """
    
    # retorna erro se o rótulo de entrada não possui 2 dimensões
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = criar_colormap()

    # retorna erro se o valor/índice do rótulo é maior que o valor de indice maximo permitida para a lista de cores.
    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]

In [38]:
FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1) 
FULL_COLOR_MAP = label_to_color_image(FULL_LABEL_MAP) 

In [39]:
class ModeloDeepLab(object):
    """Classe para carregar o modelo deeplab e fazer a inferência."""

    FROZEN_GRAPH_NAME = 'frozen_inference_graph'

    def __init__(self, tarball_path):
        """ Cria e carrega o model deeplab pré-treinado. """
        self.graph = tf.Graph()
        graph_def = None

        # Extrai os frozen graph do arquivo tar.
        tar_file = tarfile.open(tarball_path)
        for tar_info in tar_file.getmembers():
            if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
                file_handle = tar_file.extractfile(tar_info)
                graph_def = tf.compat.v1.GraphDef.FromString(file_handle.read())
                break
        tar_file.close()

        if graph_def is None:
            raise RuntimeError('Nao foi possivel encontrar o inference graph no arquivo tar.')

        with self.graph.as_default():
            tf.import_graph_def(graph_def, name='')

        self.sess = tf.compat.v1.Session(graph=self.graph)

    def run(self, image, INPUT_TENSOR_NAME = 'ImageTensor:0', OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'):
        """Roda a inferência em uma imagem. 

        Parâmetros: 
            image: Objeto PIL.Image que contém a imagem de input. 
            INPUT_TENSOR_NAME: O nome do tensor de entrada. padrão=ImageTensor
            OUTPUT_TENSOR_NAME: O name do tensor de saída. padrão=SemanticPredictions

        Returns:
            resized_image: imagem de entrada RGB redimensionada  
            seg_map: Mapa de segmentação do `resized_image` 
        """
        width, height = image.size
        target_size = (2049,1025)  # tamanho das imagens do dataset Cityscapes
        resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)

        batch_seg_map = self.sess.run(
            OUTPUT_TENSOR_NAME,
            feed_dict={INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
        
        seg_map = batch_seg_map[0]  # espera o batch size = 1
        if len(seg_map.shape) == 2:
            seg_map = np.expand_dims(seg_map,-1)  # adiciona uma dimensão extra, necessária pro cv2.resize

        seg_map = cv2.resize(seg_map, (width,height), interpolation=cv2.INTER_NEAREST)
        return seg_map

TARBALL_NAME = 'deeplab_model.tar.gz'
model_dir = tempfile.mkdtemp()
tf.io.gfile.makedirs(model_dir)
MODEL_NAME = 'mobilenetv2_coco_cityscapes_trainfine'
#MODEL_NAME = 'xception65_cityscapes_trainfine'
DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'
MODEL_URLS = {
    'mobilenetv2_coco_cityscapes_trainfine' : 'deeplabv3_mnv2_cityscapes_train_2018_02_05.tar.gz',
    'xception65_cityscapes_trainfine'       : 'deeplabv3_cityscapes_train_2018_02_06.tar.gz',
}
download_path = os.path.join(model_dir, TARBALL_NAME)
download_path
urllib.request.urlretrieve(DOWNLOAD_URL_PREFIX + MODEL_URLS[MODEL_NAME], download_path)

('/tmp/tmp8ili4xt_/deeplab_model.tar.gz',
 <http.client.HTTPMessage at 0x7faa2ece0b50>)

In [40]:
model = ModeloDeepLab(download_path)
import time

In [67]:
kernel_size = 5  
tamanho_fonte = 0.02  # 2% em realação a largura fo frame
posição_Y =0.09 # 9% em realação ao topo do frame distancia da borda
posicao_linha_esquerda = 0.30 # 80% em ao eixo x esquerda opara direita
percent_reducao =1 # Percentual para redução da imagem para processamento mair rapido 100% equivele a 1 , 0.6 60% de reducao
qtd_pessoas_id = 0
m_obj_id = 0
m_qtd_pessoas_id = 0
cap = cv2.VideoCapture('/content/people6.mp4')
subtracao = cv2.createBackgroundSubtractorMOG2()
arrayw = []
arrayh = []
frames = []
frame_count = 0 


In [103]:
def configs_video(video_largura, video_altura, nome_arquivo='resultado.avi'): 
    fourcc = cv2.VideoWriter_fourcc(*'XVID') 
    fps = 24
    saida_video = cv2.VideoWriter(nome_arquivo, fourcc, fps, (video_largura, video_altura))
    return saida_video

def exibir_video(nome_arquivo, largura=720, altura=480):
    import io
    import base64
    from IPython.display import HTML
    video_encoded = base64.b64encode(io.open(nome_arquivo, 'rb').read())
    return HTML(data='''<video width="{0}" height="{1}" alt="Video teste" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(largura, altura, video_encoded.decode('ascii')))

largura_imagem = 640  # Defina a largura desejada para as imagens
altura_imagem = 630
video_saida = configs_video(largura_imagem * 3, altura_imagem * 3, 'resultado.avi')

In [238]:
kernel_size = 5  
tamanho_fonte = 0.02  # 2% em realação a largura fo frame
posição_Y =0.09 # 9% em realação ao topo do frame distancia da borda
posicao_linha_esquerda = 0.25 # 80% em ao eixo x esquerda opara direita
percent_reducao =1 # Percentual para redução da imagem para processamento mair rapido 100% equivele a 1 , 0.6 60% de reducao
qtd_pessoas_id = 0
m_obj_id = 0
m_qtd_pessoas_id = 0
cap = cv2.VideoCapture('/content/people6.mp4')
subtracao = cv2.createBackgroundSubtractorMOG2()
arrayw = []
arrayh = []
frames = []
image_arrays = []
offset =10
frame_count = 0 
MIN_DISTANCE = 50  # pixels
DETECTION_DELAY = 1.0  # segundo

video_largura = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Largura do vídeo
video_altura = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Altura do vídeo
saida_video = configs_video(video_largura, video_altura)


while True:
    ret, frame = cap.read()
    if not ret:
        break
    frameOriginal=frame
    if frame_count < 80:
        frame_count += 1
        continue
    elif frame_count >= 90:
        break
    frame = cv2.resize(frame, None, fx=percent_reducao, fy=percent_reducao)
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    seg_map = model.run(pil_image)
    seg_map = seg_map.astype('uint8')

    linha_esquerda = int(frame.shape[1] *posicao_linha_esquerda)
    global trackers
    trackers = {}  # Chave: obj_id, Valor: (tracker, cruzou_linha)

    global obj_id
    obj_id = 0
    objetos_detectados = set() 




    def vis_segmentacaoia(image, seg_map):
        global qtd_pessoas_id, trackers, obj_id, detections, objetos_detectados
        iaseg_image = label_to_color_image(seg_map).astype(np.uint8)
        ia_person_mask = (seg_map == 11)
        ia_person_image = np.zeros_like(seg_map)
        ia_person_image[ia_person_mask] = 255
        _, ia_thresholded_image = cv2.threshold(ia_person_image, 127, 255, cv2.THRESH_TOZERO)
        ia_contours, _ = cv2.findContours(ia_thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contornos_e_areas = []

        for ia_contour in ia_contours:
            ia_x, ia_y, ia_w, ia_h = cv2.boundingRect(ia_contour)
            ia_area = ia_w * ia_h 
            contornos_e_areas.append((ia_contour, ia_area, ia_w, ia_h))

        contornos_e_areas.sort(key=lambda x: x[1], reverse=True)
        max_area = contornos_e_areas[0][1]

        for ia_contour, ia_area, ia_w, ia_h in contornos_e_areas:
            if ia_area >= max_area * 0.5 and ia_h > ia_w * 1.3:  
                ia_x, ia_y, ia_w, ia_h = cv2.boundingRect(ia_contour)
                tracker = cv2.TrackerCSRT_create()  # Alterado para o rastreador CSRT
                tracker.init(image, (ia_x, ia_y, ia_w, ia_h))
                trackers[obj_id] = (tracker, False)
                obj_id += 1

        detections = []
        for id, (tracker, cruzou_linha) in trackers.items():
            success, box = tracker.update(image)

            if success:
                ia_x, ia_y, ia_w, ia_h = map(int, box)
                cv2.rectangle(iaseg_image, (ia_x, ia_y), (ia_x + ia_w, ia_y + ia_h), (255, 255, 255), 2)
                centro_x = ia_x + ia_w // 2
                centro_y = ia_y + ia_h // 2
                cv2.circle(iaseg_image, (centro_x, centro_y), 10, (255, 0, 0), -1)
                cv2.line(iaseg_image, (linha_esquerda, 0), (linha_esquerda, iaseg_image.shape[0]), (255, 255, 255), 5)

                if not cruzou_linha and abs(centro_x - linha_esquerda) <= offset:
                    current_time = time.time()
                    recent_detection = next((d for d in detections if d['time'] > current_time - DETECTION_DELAY and
                    abs(d['position'][0] - centro_x) < MIN_DISTANCE and abs(d['position'][1] - centro_y) < MIN_DISTANCE), None)

                    if not recent_detection:
                        # Verifica se o objeto já foi detectado anteriormente
                        if id not in objetos_detectados:
                            # Adiciona o ID do objeto ao conjunto de objetos detect
                            objetos_detectados.add(id) 
                            # Se não, adiciona a detecção à lista e                        # Se não, adiciona a detecção à lista e aumenta a contagem de pessoas
                        detections.append({'position': (centro_x, centro_y), 'time': current_time})
                        trackers[id] = (tracker, True)
                        qtd_pessoas_id += 1

        font = cv2.FONT_HERSHEY_SIMPLEX
        text = "Quantidade de pessoas IA = {}".format(qtd_pessoas_id)
        # Define o tamanho da fonte como 4% da largura da imagem
        fontScale = iaseg_image.shape[1] * tamanho_fonte / cv2.getTextSize("X", font, 1, 2)[0][0]
        textsize = cv2.getTextSize(text, font, fontScale, 2)[0]
        textX = (iaseg_image.shape[1] - textsize[0]) // 2
        textY = int(iaseg_image.shape[0] * posição_Y )
        cv2.putText(iaseg_image, text, (textX, textY), font, fontScale, (255, 255, 255), 2, cv2.LINE_AA)

        resized_frame = cv2.resize(image, (iaseg_image.shape[1], iaseg_image.shape[0]))
        overlay = cv2.addWeighted(resized_frame, 0.3, iaseg_image, 0.7, 0)
        return overlay






    imagem_sobreposta = vis_segmentacaoia(frame, seg_map)


    fg_mask = subtracao.apply(frame)
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    opening = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    dilat = cv2.dilate(closing, np.ones((10, 10)))
    dilatacao = cv2.dilate(dilat, np.ones((3, 3), np.uint8))
    contours, hierarchy = cv2.findContours(dilatacao, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, thresholded_frame = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_TOZERO)
    _, inverted_frame = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_TOZERO_INV)

        
    areas = []  
    maior_area = 0  

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = w * h
        areas.append(area)
        for area in areas:
            if area > maior_area:
                maior_area = area
    if areas:  
        print(maior_area)
    for contour, area in zip(contours, areas):
        x, y, w, h = cv2.boundingRect(contour)
        if h >= w * 1.3 and area >= maior_area * 0.5:
            #cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            tracker = cv2.TrackerCSRT_create()  
            tracker.init(frame, (x, y, w, h))
            trackers[m_obj_id] = (tracker, False)
            m_obj_id += 1

    def conta_pessoas(deteccoes):
        global m_qtd_pessoas_id
        novas_deteccoes = []
        for deteccao in deteccoes:
            x, y = deteccao['x'], deteccao['y']
            if abs(x - linha_esquerda) <= offset:
                m_qtd_pessoas_id += 1
                cv2.line(frame, (linha_esquerda, 0), (linha_esquerda, frame.shape[0]), (0, 127, 255), 3)
                print("Pessoas detectadas até o momento: " + str(m_qtd_pessoas_id))
            else:
                novas_deteccoes.append(deteccao)
        return novas_deteccoes

    m_detections = []
    for id, (tracker, cruzou_linha) in trackers.items():
        success, box = tracker.update(frame)
        if success:
            x, y, w, h = map(int, box)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            centro_x = x + w // 2
            centro_y = y + h // 2
            cv2.circle(frame, (centro_x, centro_y), 10, (0, 0, 255), -1)
            cv2.line(frame, (linha_esquerda, 0), (linha_esquerda, frame.shape[0]), (255, 0, 0), 5) 
            if not cruzou_linha and abs(centro_x - linha_esquerda) <= offset:
                current_time = time.time()
                recent_detection = next((d for d in m_detections if d['time'] > current_time - DETECTION_DELAY and abs(d['x'] - centro_x) < MIN_DISTANCE and abs(d['y'] - centro_y) < MIN_DISTANCE), None)
                if not recent_detection:
                    m_detections.append({'x': centro_x, 'y': centro_y, 'time': current_time})
                trackers[id] = (tracker, True)
    m_detections = conta_pessoas(m_detections)

    font = cv2.FONT_HERSHEY_SIMPLEX
    text = "Quantidade de pessoas = {}".format(m_qtd_pessoas_id)
    fontScale = frame.shape[1] * tamanho_fonte / cv2.getTextSize("X", font, 1, 2)[0][0]
    textsize = cv2.getTextSize(text, font, fontScale, 2)[0]
    textX = (frame.shape[1] - textsize[0]) // 2
    textY = int(frame.shape[0] * posição_Y )
    cv2.putText(frame, text, (textX, textY), font, fontScale, (255, 255, 255), 2, cv2.LINE_AA)
    frames.append(frame)

    ret, _ = cap.read()
    if ret:
        clear_output(wait=True)  
  
    def vis_segmentacao(image, seg_map):
        seg_image = label_to_color_image(seg_map).astype(np.uint8)
        fig = plt.figure(figsize=(30, 20))
        gs = GridSpec(4, 3, figure=fig)
        axes = [fig.add_subplot(gs[i//3, i%3]) for i in range(9)]
        ax10 = fig.add_subplot(gs[3, :]) 

        images = [cv2.cvtColor(frameOriginal, cv2.COLOR_BGR2RGB),cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)  , cv2.cvtColor(thresholded_frame, cv2.COLOR_GRAY2RGB)  ,cv2.cvtColor(opening, cv2.COLOR_GRAY2RGB) , cv2.cvtColor(dilat, cv2.COLOR_GRAY2RGB) , 
              image, cv2.cvtColor(frameOriginal, cv2.COLOR_BGR2RGB), seg_image, imagem_sobreposta ]

        titles = ['Imagem Original', 'Imagem gray_frame', 'Imagem thresholded_frame',
                  'Imagem opening', 'Imagem dilatacao', 'Imagem Segmentacao Manual',
                  'Original', 'Mapa de segmentação', 'Sobreposição da segmentação']



        for ax, img, title in zip(axes, images, titles):
                if title == 'Imagem thresholded_frame':
                    ax.imshow(img, cmap='gray')  # Plota a imagem com colormap 'gray'
                else:
                    ax.imshow(img)
                ax.axis('off')
                ax.set_title(title)
            
                unique_labels = np.unique(seg_map)
                ax10.imshow(FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
                ax10.set_yticks(range(len(unique_labels)))
                ax10.set_yticklabels(LABEL_NAMES[unique_labels])
                ax10.set_xticks([])
                ax10.grid(False)
                ax10.set_title('Mapa de cores')
            # Convertendo a imagem do subplot em um array e adicionando à lista
                image_arrays.append(np.array(ax.images[0].get_array()))
      
        plt.tight_layout()
        plt.show()
        return image_arrays  # retornar a lista de arrays de imagem

    image_arrays = vis_segmentacao(frame, seg_map)

    frames.append(frame)
    frame_count += 1  


fig, axs = plt.subplots(len(image_arrays), figsize=(40, len(image_arrays)))

for ax, img in zip(axs, image_arrays):
    ax.imshow(img)
    ax.axis('off')

plt.tight_layout()
plt.show()

   
   
cap.release()



KeyboardInterrupt: ignored

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [231]:
import cv2
import numpy as np

# Suponha que image_arrays seja uma lista de suas imagens e que haja um múltiplo de 9 delas
assert len(image_arrays) % 9 == 0

# Converta todas as imagens para o formato BGR que o OpenCV usa
image_arrays_bgr = [cv2.cvtColor(img, cv2.COLOR_RGB2BGR) for img in image_arrays]

# Verifique o tamanho das imagens
height, width, _ = image_arrays_bgr[0].shape

# Criando um vídeo de 3x3 imagens
frame_size = (width * 3, height * 3)  # Cada quadro terá 3 imagens de largura e 3 imagens de altura
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 24
out = cv2.VideoWriter('output2.mp4', fourcc, 30.0, frame_size)
nome_arquivo='output2.mp4'
for i in range(0, len(image_arrays_bgr), 9):  # Processando 9 imagens de cada vez
    # Criando uma grade de imagens 3x3
    grid_image = []
    for j in range(3):  # 3 linhas
        row_image = cv2.hconcat(image_arrays_bgr[i+3*j:i+3*j+3])  # Concatenando 3 imagens para fazer uma linha
        grid_image.append(row_image)
    grid_image = cv2.vconcat(grid_image)  # Concatenando as linhas para fazer a grade completa

    out.write(grid_image)

out.release()

In [232]:
def exibir_video_base64(nome_arquivo, largura=720, altura=480):
  import io
  import base64
  from IPython.display import HTML

  # Abrir o arquivo de vídeo em modo binário
  with open(nome_arquivo, 'rb') as f:
    video = f.read()
 # !ffmpeg -y -loglevel panic -i output2.mp4 resultado_driveseg.mp4
  # Converter o vídeo em uma string Base64
  video_base64 = base64.b64encode(video).decode('ascii')

  # Construir a tag HTML para o vídeo
  video_tag = '''
  <video width="{}" height="{}" controls>
    <source src="data:video/mp4;base64,{}" type="video/mp4">
  </video>
  '''.format(largura, altura, video_base64)

  # Retornar a tag HTML
  return HTML(data=video_tag)

In [233]:
!ffmpeg -y -loglevel panic -i output2.mp4 resultado_driveseg.mp4

In [234]:
exibir_video_base64('resultado_driveseg.mp4')